In [1]:
import math
import numpy as np

def simulate_buffon_vectorized(needle_length, line_distance, n_drops, chunk_size=10**7):
    """
    Perform a vectorized Buffon's Needle simulation using chunking.

    Parameters:
        needle_length (float): The length of the needle.
        line_distance (float): The distance between the parallel lines.
        n_drops (int): Total number of needle drops.
        chunk_size (int): Number of drops processed in one chunk (default: 10^7).

    Returns:
        total_hits (int): Total number of times the needle crosses a line.
    """
    total_hits = 0
    rng = np.random.default_rng()  # use the new numpy random Generator
    remaining = n_drops

    # Process drops in chunks to avoid memory issues
    while remaining > 0:
        current_chunk = min(chunk_size, remaining)

        # For the short needle method, we sample:
        #   - x: distance from needle's center to the closest line, uniformly in [0, d/2]
        #   - theta: angle in [0, pi/2]
        xs = rng.uniform(0, line_distance / 2, current_chunk)
        thetas = rng.uniform(0, np.pi / 2, current_chunk)

        # The needle crosses a line if:
        #   x <= (needle_length/2) * sin(theta)
        hits = np.count_nonzero(xs <= (needle_length / 2) * np.sin(thetas))
        total_hits += hits
        remaining -= current_chunk

    return total_hits

def refined_simulation_vectorized(needle_length, line_distance, n_drops, chunk_size=10**7):
    """
    Run the optimized simulation and compute the observed and theoretical probabilities,
    as well as an approximation for π.

    Parameters:
        needle_length (float): Length of the needle (l).
        line_distance (float): Distance between lines (d).
        n_drops (int): Total number of needle drops (N).
        chunk_size (int): Number of drops processed per chunk.

    Returns:
        observed_prob (float): The observed crossing probability.
        theo_prob (float): The theoretical crossing probability.
        approx_pi (float): The π approximation from the simulation.
    """
    hits = simulate_buffon_vectorized(needle_length, line_distance, n_drops, chunk_size)
    observed_prob = hits / n_drops

    # Compute theoretical probability using known formulas.
    if needle_length <= line_distance:
        theo_prob = (2 * needle_length) / (line_distance * math.pi)
    else:
        term1 = (needle_length - math.sqrt(needle_length**2 - line_distance**2)) / line_distance
        term2 = math.acos(line_distance / needle_length)
        theo_prob = (2 / math.pi) * (term1 + term2)

    # Compute the approximation for π.
    if hits == 0:
        approx_pi = None  # Avoid division by zero.
    else:
        if needle_length <= line_distance:
            approx_pi = (2 * needle_length * n_drops) / (line_distance * hits)
        else:
            approx_pi = (2 * n_drops / hits) * (term1 + term2)

    print("\n--- Optimized Vectorized Simulation Results ---")
    print(f"Needle length (l): {needle_length}")
    print(f"Distance between lines (d): {line_distance}")
    print(f"Number of drops (N): {n_drops}")
    print(f"Number of hits (needle crosses a line): {hits}")
    print(f"Observed crossing probability: {observed_prob:.6f}")
    print(f"Theoretical crossing probability: {theo_prob:.6f}")

    if approx_pi is None:
        print("No needle crossed a line; cannot approximate π.")
    else:
        print(f"Approximated π: {approx_pi:.6f}")

    return observed_prob, theo_prob, approx_pi

def main():
    print("Optimized Buffon's Needle Simulation")
    # Get simulation parameters from the user.
    needle_length = float(input("Enter the needle length (l): "))
    line_distance = float(input("Enter the distance between lines (d): "))
    n_drops = int(input("Enter the total number of drops (N): "))
    chunk_size = int(input("Enter the chunk size for simulation (e.g., 10000000): "))

    # Run the simulation.
    refined_simulation_vectorized(needle_length, line_distance, n_drops, chunk_size)

if __name__ == "__main__":
    main()
import math
import numpy as np
import matplotlib.pyplot as plt

def simulate_buffon_antithetic(needle_length, line_distance, n_drops, chunk_size=10**7):
    """
    Perform a vectorized Buffon's Needle simulation using antithetic sampling.

    In antithetic sampling, for each random drop, we also simulate its "mirror image"
    (with respect to the uniform variable used for theta). This typically produces a pair of
    negatively correlated estimates, reducing overall variance.

    Parameters:
        needle_length (float): The length of the needle.
        line_distance (float): The distance between the parallel lines.
        n_drops (int): Total number of needle drops.
        chunk_size (int): Number of drops processed in one chunk.

    Returns:
        total_hits (int): Total number of times the needle crosses a line.
    """
    total_hits = 0
    rng = np.random.default_rng()
    remaining = n_drops

    while remaining > 0:
        current_chunk = min(chunk_size, remaining)

        # Sample x and theta for each drop:
        xs = rng.uniform(0, line_distance / 2, current_chunk)
        thetas = rng.uniform(0, np.pi / 2, current_chunk)
        # Create antithetic variates for theta:
        thetas_antithetic = np.pi/2 - thetas

        # Compute the crossing conditions for the original and antithetic samples.
        hits_original = xs <= (needle_length / 2) * np.sin(thetas)
        hits_antithetic = xs <= (needle_length / 2) * np.sin(thetas_antithetic)
        hits = np.count_nonzero(hits_original) + np.count_nonzero(hits_antithetic)

        total_hits += hits
        remaining -= current_chunk

    # Because each drop now gives two "samples", the effective number of drops is 2*n_drops.
    return total_hits, 2 * n_drops

def simulate_buffon_stratified(needle_length, line_distance, n_drops, n_strata=5):
    """
    Perform a stratified Buffon's Needle simulation.

    The x variable (distance from the center to the nearest line) is stratified into n_strata
    bins in [0, d/2]. For each stratum, we sample the same number of drops. This guarantees
    that each region of the domain is equally represented, reducing the variance.

    Parameters:
        needle_length (float): The length of the needle.
        line_distance (float): The distance between lines.
        n_drops (int): Total number of drops.
        n_strata (int): Number of strata (bins) for the x variable.

    Returns:
        total_hits (int): Total number of needle crossings.
    """
    rng = np.random.default_rng()
    # Divide [0, d/2] into n_strata equal parts.
    strata_edges = np.linspace(0, line_distance/2, n_strata + 1)
    drops_per_stratum = n_drops // n_strata
    total_hits = 0
    total_effective = drops_per_stratum * n_strata

    for i in range(n_strata):
        # Sample x uniformly in the i-th stratum:
        xs = rng.uniform(strata_edges[i], strata_edges[i+1], drops_per_stratum)
        # Sample theta uniformly in [0, pi/2]
        thetas = rng.uniform(0, np.pi/2, drops_per_stratum)

        hits = np.count_nonzero(xs <= (needle_length / 2) * np.sin(thetas))
        total_hits += hits

    return total_hits, total_effective

def estimate_pi(observed_hits, total_drops, needle_length, line_distance):
    """
    Estimate pi using the formula derived from the classical Buffon needle problem:

    For a needle of length l and spacing d (with l <= d), the crossing probability is:
      P = (2*l) / (d * pi)
    Hence:
      pi ≈ (2*l * total_drops) / (d * observed_hits)
    """
    if observed_hits == 0:
        return None
    return (2 * needle_length * total_drops) / (line_distance * observed_hits)

def main():
    # Simulation parameters
    needle_length = 1.0  # l, must be <= d
    line_distance = 1.0  # d
    n_drops = 1001_000  # number of needle drops

    print("Buffon's Needle Simulation: Standard Vectorized Method")
    # Standard simulation (from your original code)
    # Here we use the refined_simulation_vectorized function defined previously.
    # (Assume that function is defined in our module as provided above.)
    # For brevity, we call it as follows:
    from time import time
    t0 = time()
    hits_standard = simulate_buffon_vectorized(needle_length, line_distance, n_drops)
    t1 = time()
    pi_standard = estimate_pi(hits_standard, n_drops, needle_length, line_distance)
    error_standard = abs(pi_standard - math.pi)
    print(f"Standard Method: Pi ~ {pi_standard:.6f} (Error: {error_standard:.6f}) (Time: {t1-t0:.2f}s)")

    print("\nBuffon's Needle Simulation: Antithetic Sampling")
    t0 = time()
    hits_antithetic, total_effective = simulate_buffon_antithetic(needle_length, line_distance, n_drops)
    t1 = time()
    pi_antithetic = estimate_pi(hits_antithetic, total_effective, needle_length, line_distance)
    error_antithetic = abs(pi_antithetic - math.pi)
    print(f"Antithetic Sampling: Pi ~ {pi_antithetic:.6f} (Error: {error_antithetic:.6f}) (Effective drops: {total_effective}, Time: {t1-t0:.2f}s)")


    print("\nBuffon's Needle Simulation: Stratified Sampling")
    t0 = time()
    hits_stratified, effective_strata = simulate_buffon_stratified(needle_length, line_distance, n_drops, n_strata=20)
    t1 = time()
    pi_stratified = estimate_pi(hits_stratified, effective_strata, needle_length, line_distance)
    error_stratified = abs(pi_stratified - math.pi)
    print(f"Stratified Sampling: Pi ~ {pi_stratified:.6f} (Error: {error_stratified:.6f}) (Effective drops: {effective_strata}, Time: {t1-t0:.2f}s)")



if __name__ == "__main__":
    main()
# Conttrol variate Reduction Technique next

Optimized Buffon's Needle Simulation
Enter the needle length (l): 1
Enter the distance between lines (d): 1
Enter the total number of drops (N): 10000
Enter the chunk size for simulation (e.g., 10000000): 1000

--- Optimized Vectorized Simulation Results ---
Needle length (l): 1.0
Distance between lines (d): 1.0
Number of drops (N): 10000
Number of hits (needle crosses a line): 6438
Observed crossing probability: 0.643800
Theoretical crossing probability: 0.636620
Approximated π: 3.106555
Buffon's Needle Simulation: Standard Vectorized Method
Standard Method: Pi ~ 3.141338 (Error: 0.000254) (Time: 0.21s)

Buffon's Needle Simulation: Antithetic Sampling
Antithetic Sampling: Pi ~ 3.140444 (Error: 0.001149) (Effective drops: 2002000, Time: 0.30s)

Buffon's Needle Simulation: Stratified Sampling
Stratified Sampling: Pi ~ 3.147452 (Error: 0.005860) (Effective drops: 1001000, Time: 0.09s)
